In [26]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import json
from sqlalchemy import text
import ast

In [2]:
df = pd.read_parquet('c:/Users/aksha/OneDrive/Desktop/CARS-FINAL_YEAR_PROJECT/DATA/04_After_data_preprocessing/webdev1.parquet')

## Dropping unnecessary cols

All the columns that are ranging from Adaptive Cruise control to Third Row Seating will be dropped, as there are only True values. For the car which have these col value as NaN, may or may not have this Features, and in future if I give option to add more cars to users, user may give True as well as False value, which will clutter the entire data (some rows have true and false , some have only true but NaN can represent False).

In [3]:
del_cols = df.loc[:, 'Adaptive_Cruise_Control' : 'Third_Row_Seating'].columns

df.drop(columns=del_cols, inplace=True)

## Rating and Open Recall

- Open Recall have only one category which is `not_in_use_yet` and all other are NaN, which do not make sense to keep.
- Rating and People who gave rating to car or seller will be deleted.

In [4]:
df.drop(columns=['Seller_Rating', 'Open_Recall',
       'People_Count_Seller_Rating', 'Car_Rating',
       'People_Count_Car_Rating'], inplace=True)

In [5]:
df.rename(columns={'Km/L_e_City' : 'Km_L_e_City', 'Km/L_e_Hwy' : 'Km_L_e_Hwy', 'Km/L' : 'Km_per_l'}, inplace=True)

In [6]:
# basic imputation techniques

df = df.assign(
    Dc_Fast_Charging = np.where(
        df['Fuel_Type'] != 'Electric',
        0,
        df['Dc_Fast_Charging']
    ),

    Level2_Charging = np.where(
        df['Fuel_Type'] != 'Electric',
        0,
        df['Level2_Charging']
    ),

    Battery_Capacity = np.where(
        df['Fuel_Type'] != 'Electric',
        0,
        df['Battery_Capacity']
    ),

    Expected_Range = np.where(
        df['Fuel_Type'] != 'Electric',
        0,
        df['Expected_Range']
    ),

    Km_L_e_City = np.where(
        df['Fuel_Type'] != 'Electric',
        0,
        df['Km_L_e_City']
    ),

    Km_L_e_Hwy = np.where(
        df['Fuel_Type'] != 'Electric',
        0,
        df['Km_L_e_Hwy']
    ),

    Km_per_l = np.where(
        df['Fuel_Type'] == 'Electric',
        0,
        df['Km_per_l']
    ),

    Valves = np.where(
        df['Fuel_Type'] == 'Electric',
        0,
        df['Valves']
    ),


    Cylinder_Config = np.where(
        df['Fuel_Type'] == 'Electric',
        0,
        df['Cylinder_Config']
    ),

    Engine_Size	 = np.where(
        df['Fuel_Type'] == 'Electric',
        0,
        df['Engine_Size']
    ),

    Gear_Spec	 = np.where(
        df['Fuel_Type'] == 'Electric',
        0,
        df['Gear_Spec']
    )
)

In [7]:
df = df.assign(
    Drivetrain = np.where(
        df['Drivetrain'].isna(),
        df['Drivetrain'].mode().values[0],
        df['Drivetrain']
    ),

    Km_per_l = np.select(
      condlist=  [
            df['Fuel_Type'] == 'Electric',
            df['Km_per_l'].isna()
        ],
      choicelist= [
          0,
          df['Km_per_l'].median()
      ],
      default=df['Km_per_l']
    ),

    Dc_Fast_Charging = np.select(
        condlist= [
            df['Fuel_Type'] != 'Electric',
            df['Dc_Fast_Charging'].isna()
        ],
        choicelist = [
            0,
            df['Dc_Fast_Charging'].median()
        ],
        default=df['Dc_Fast_Charging']
    ),

    Battery_Capacity = np.select(
        condlist= [
            df['Fuel_Type'] != 'Electric',
            df['Battery_Capacity'].isna()
        ],
        choicelist = [
            0,
            df['Battery_Capacity'].median()
        ],
        default=df['Battery_Capacity']
    ),

    Expected_Range = np.select(
        condlist= [
            df['Fuel_Type'] != 'Electric',
            df['Expected_Range'].isna()
        ],
        choicelist = [
            0,
            df['Expected_Range'].median()
        ],
        default=df['Expected_Range']
    ),

    Gear_Spec = np.where(
        df['Gear_Spec'].isna(),
        df['Gear_Spec'].mode().values[0],
        df['Gear_Spec']
    ),

    Engine_Size = np.select(
        condlist= [
            df['Fuel_Type'] == 'Electric',
            df['Engine_Size'].isna()
        ],
        choicelist = [
            0,
            df['Engine_Size'].median()
        ],
        default=df['Engine_Size']
    ),

    Cylinder_Config = np.select(
        condlist= [
            df['Fuel_Type'] == 'Electric',
            df['Cylinder_Config'].isna()
        ],
        choicelist = [
            0,
            df['Cylinder_Config'].mode().values[0]
        ],
        default=df['Cylinder_Config']
    ),

    Valves = np.select(
        condlist= [
            df['Fuel_Type'] == 'Electric',
            df['Valves'].isna()
        ],
        choicelist = [
            0,
            df['Valves'].mode().values[0]
        ],
        default=df['Valves']
    ),

    Km_L_e_City = np.select(
        condlist= [
            df['Fuel_Type'] != 'Electric',
            df['Km_L_e_City'].isna()
        ],
        choicelist = [
            0,
            df['Km_L_e_City'].median()
        ],
        default=df['Km_L_e_City']
    ),

    Km_L_e_Hwy = np.select(
        condlist= [
            df['Fuel_Type'] != 'Electric',
            df['Km_L_e_Hwy'].isna()
        ],
        choicelist = [
            0,
            df['Km_L_e_Hwy'].median()
        ],
        default=df['Km_L_e_Hwy']
    )
)

In [8]:
# I will drop only those NaN values which belong to following cols, as this cols will be used in Building Recommendation system.

cols = ['Brand_Name','Stock_Type','Drivetrain','Fuel_Type','One_Owner_Vehicle',
 'Personal_Use_Only','Gear_Spec','Cylinder_Config','Valves','ST',
 'Model_Name','Seller_Name','Model_Year','Mileage','Price','Km_per_l','Dc_Fast_Charging','Battery_Capacity',
 'Expected_Range','Engine_Size','Km_L_e_City','Km_L_e_Hwy']

In [9]:
df = df.dropna(subset=cols)

In [10]:
# correcting the indecies

df.reset_index(drop=True, inplace=True)

In [11]:
# creating index col, which will be used as primary key in mysql db
df.insert(0, 'index', range(1,df.shape[0]+1))

In [12]:
df['Image_List'] = df['Image_List'].apply(lambda x : ast.literal_eval(x) if x else x)

In [13]:
df['Image_List'] = df['Image_List'].apply(lambda x : json.dumps(x) if x else x)

In [69]:
df.rename(columns={'index' : 'id'}, inplace=True)

In [ ]:
username = 'akshat'
password = 'papadukangae123'
hostname = 'autonexus-webdev-data.c1y6s6k0csni.ap-south-1.rds.amazonaws.com'
port = '3306'


engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{hostname}:{port}"
)

In [37]:
with engine.connect() as conn:
    conn.execute(text('CREATE DATABASE autonexus;'))
    conn.commit()

In [76]:
engine = create_engine(
    f"mysql+pymysql://{username}:{password}@{hostname}:{port}/autonexus"
)

In [70]:
df.to_sql('Car',con=engine, chunksize=500, index=False)

98757

In [77]:
with engine.connect() as conn:
    conn.execute(text('update Car set `id` = `id` -1;'))
    conn.commit()